In [ ]:
import pandas as pd
import glob
import os
import json
from anarci import anarci
from tqdm.notebook import tqdm

In [58]:
def load_asd_data_with_pandas(data_path: str) -> pd.DataFrame:
    """
    Загружает все parquet файлы из папки asd в один pandas DataFrame

    Args:
        data_path: путь к папке с данными

    Returns:
        pd.DataFrame: объединенный DataFrame со всеми данными
    """
    # Получаем все parquet файлы из папки
    parquet_files = glob.glob(os.path.join(data_path, "part-*.parquet"))

    if not parquet_files:
        raise ValueError(f"Не найдено parquet файлов в папке {data_path}")

    print(f"Найдено {len(parquet_files)} parquet файлов")

    # Загружаем все файлы в список DataFrame'ов
    dataframes = []
    for file_path in parquet_files:
        df = pd.read_parquet(file_path)
        dataframes.append(df)

    # Объединяем все DataFrame'ы в один
    combined_df = pd.concat(dataframes, ignore_index=True)

    print(f"Общий размер данных: {combined_df.shape}")
    print(f"Колонки: {list(combined_df.columns)}")

    return combined_df

# Загружаем данные
agab_df = load_asd_data_with_pandas('./asd')

Найдено 20 parquet файлов
Общий размер данных: (1227083, 11)
Колонки: ['dataset', 'heavy_sequence', 'light_sequence', 'scfv', 'affinity_type', 'affinity', 'antigen_sequence', 'confidence', 'nanobody', 'metadata', 'processed_measurement']


In [59]:
# Переводим affinity и processed_measurement во float где возможно
agab_df = agab_df.copy()
agab_df.loc[:, 'affinity'] = pd.to_numeric(agab_df['affinity'], errors='coerce').fillna(agab_df['affinity'])
agab_df.loc[:, 'processed_measurement'] = pd.to_numeric(agab_df['processed_measurement'], errors='coerce').fillna(agab_df['processed_measurement'])
type(agab_df['affinity'][agab_df['affinity_type'] == 'bool'].iloc[0])

float

#### Base filtering

In [60]:
is_not_nanobody = agab_df['nanobody'] == False
# is_high_confidence = agab_df['confidence'].isin(['high', 'very_high'])
is_scfv = agab_df['scfv'] == True
has_both_chains = (
    agab_df['light_sequence'].notna() 
    & agab_df['heavy_sequence'].notna()
    & (agab_df['light_sequence'] != '')
    & (agab_df['heavy_sequence'] != '')
)

agab_df = agab_df[is_not_nanobody 
# & is_high_confidence 
& (is_scfv | has_both_chains)]
agab_df.shape

(844872, 11)

In [61]:
agab_df.groupby(['affinity_type', 'dataset'])['antigen_sequence'].agg([
    ('total_count', 'count'),
    ('unique_antigens', 'nunique')
]).sort_values('unique_antigens', ascending=False)

total_count  unique_antigens
affinity_type         dataset                                              
bool                  patents                        21621             3478
                      structures-antibodies           2711             1083
                      genbank                           98               23
                      skempiv2                          34               21
ddg                   skempiv2                         400               19
elisa_mut_to_wt_ratio abdesign                         658               13
bool                  abdesign                          14               13
                      ab-bind                           13               10
ic_50                 dlgo                             360               10
ddg                   ab-bind                          270                9
-log KD               abbd                          152401                5
kd                    flab_hie2022                      55                3
alphaseq              alphaseq                      131645                2
kd                    flab_rosace2023                   25                2
log_enrichment        abbd                            3452                2
fuzzy                 buzz                          524346                1
kd                    flab_koenig2017                 4275                1
                      flab_shanehsazzadeh2023          446                1
                      flab_warszawski2019             2048                1

In [62]:
# (порог, оператор): '<', '>', '=='
AFFINITY_THRESHOLDS = {
    'fuzzy': ('h', '=='),
    'bool': (1, '=='),
    'alphaseq': (2, '<'),
    '-log KD': (7, '>'),
    'kd': (100, '<'),
    'delta_g': (-9.5, '<'),
    'log_enrichment': (1, '>'),
    'elisa_mut_to_wt_ratio': (1, '>'),
    'ic_50': (100, '<'),
}

def apply_affinity_filter(df: pd.DataFrame, thresholds: dict) -> pd.DataFrame:
    masks = []
    for affinity_type, (threshold, op) in thresholds.items():
        type_mask = df['affinity_type'] == affinity_type
        if op == '==':
            mask = type_mask & (df['processed_measurement'] == threshold)
        else:
            numeric_affinity = pd.to_numeric(df['processed_measurement'], errors='coerce')
            if op == '<':
                mask = type_mask & (numeric_affinity < threshold)
            else:
                mask = type_mask & (numeric_affinity > threshold)
        masks.append(mask)

    if masks:
        combined_mask = masks[0]
        for mask in masks[1:]:
            combined_mask = combined_mask | mask
        return df[combined_mask]
    else:
        return df

print(f"После базовых фильтров: {agab_df.shape}")
print(f"Распределение по affinity_type до фильтрации по порогам:")
print(agab_df['affinity_type'].value_counts())

# Применяем фильтрацию по порогам аффиности
agab_df = apply_affinity_filter(agab_df, AFFINITY_THRESHOLDS)

print(f"\nПосле фильтрации по порогам аффинитета: {agab_df.shape}")
print(f"Распределение по affinity_type после фильтрации:")
print(agab_df['affinity_type'].value_counts())


После базовых фильтров: (844872, 11)
Распределение по affinity_type до фильтрации по порогам:
affinity_type
fuzzy                    524346
-log KD                  152401
alphaseq                 131645
bool                      24491
kd                         6849
log_enrichment             3452
ddg                         670
elisa_mut_to_wt_ratio       658
ic_50                       360
Name: count, dtype: int64

После фильтрации по порогам аффинитета: (402392, 11)
Распределение по affinity_type после фильтрации:
affinity_type
fuzzy                    172149
-log KD                  141805
alphaseq                  55670
bool                      24491
kd                         6744
log_enrichment             1016
ic_50                       349
elisa_mut_to_wt_ratio       168
Name: count, dtype: int64


In [63]:
agab_df.groupby(['affinity_type', 'dataset'])['antigen_sequence'].agg([
    ('total_count', 'count'),
    ('unique_antigens', 'nunique')
]).sort_values('unique_antigens', ascending=False)

total_count  unique_antigens
affinity_type         dataset                                              
bool                  patents                        21621             3478
                      structures-antibodies           2711             1083
                      genbank                           98               23
                      skempiv2                          34               21
                      abdesign                          14               13
elisa_mut_to_wt_ratio abdesign                         168               12
ic_50                 dlgo                             349               10
bool                  ab-bind                           13               10
-log KD               abbd                          141805                5
kd                    flab_hie2022                      55                3
alphaseq              alphaseq                       55670                2
kd                    flab_rosace2023                   25                2
fuzzy                 buzz                          172149                1
kd                    flab_koenig2017                 4275                1
                      flab_shanehsazzadeh2023          341                1
                      flab_warszawski2019             2048                1
log_enrichment        abbd                            1016                1

#### Affinity variation

In [64]:
print("=== АНАЛИЗ РАЗЛИЧИЙ В ПОЛЯХ ДЛЯ ДУБЛИКАТОВ ПО ПОСЛЕДОВАТЕЛЬНОСТЯМ (agab_df) ===\n")

# Поля для проверки
fields = [
    "dataset",
    "scfv",
    "affinity_type",
    "affinity",
    "confidence",
    "nanobody",
    "processed_measurement",
]

# --- Поиск групп дубликатов по heavy_sequence + light_sequence + antigen_sequence---
# Находим группы, где хотя бы 2 записи с одинаковой парой последовательностей
group_cols = ["heavy_sequence", "light_sequence", "antigen_sequence"]
grouped = agab_df.groupby(group_cols, sort=False)

# Маска строк, которые входят в группы размером > 1
dup_mask = grouped[group_cols[0]].transform("size") > 1
hl_duplicates = agab_df.loc[dup_mask].copy()

# Переиспользуем groupby только на дубликатах
dup_groups = hl_duplicates.groupby(group_cols, sort=False)
unique_groups_count = dup_groups.ngroups

print(f"Всего групп дубликатов: {unique_groups_count}\n")

# --- Анализ различий по полям ---

# Для каждой группы считаем, сколько уникальных значений в каждом поле
# dropna=False, чтобы различия NaN / не-NaN тоже учитывались
nunique_per_group = dup_groups[fields].nunique(dropna=False)

# Булева матрица: True, если в группе по полю есть различия
diff_mask = nunique_per_group > 1

# Есть ли вообще различия в группе
group_has_diffs = diff_mask.any(axis=1)

groups_with_diffs = int(group_has_diffs.sum())
groups_identical = int((~group_has_diffs).sum())
total = groups_with_diffs + groups_identical if (groups_with_diffs + groups_identical) > 0 else 1

print(f"Групп с различиями: {groups_with_diffs} ({groups_with_diffs / total * 100:.2f}%)")
print(f"Групп без различий: {groups_identical} ({groups_identical / total * 100:.2f}%)\n")

# Количество строк (записей) с различиями и без различий
rows_with_diffs = group_has_diffs[group_has_diffs].index  # индексы групп с различиями
rows_without_diffs = group_has_diffs[~group_has_diffs].index  # индексы групп без различий

num_rows_with_diffs = dup_groups.size().loc[rows_with_diffs].sum() if len(rows_with_diffs) > 0 else 0
num_rows_without_diffs = dup_groups.size().loc[rows_without_diffs].sum() if len(rows_without_diffs) > 0 else 0
rows_total = num_rows_with_diffs + num_rows_without_diffs if (num_rows_with_diffs + num_rows_without_diffs) > 0 else 1

print(f"Строк (записей) с различиями: {num_rows_with_diffs} ({num_rows_with_diffs / rows_total * 100:.2f}%)")
print(f"Строк (записей) без различий: {num_rows_without_diffs} ({num_rows_without_diffs / rows_total * 100:.2f}%)\n")

# Считаем, по скольким группам отличается каждое поле
field_diffs_counts = diff_mask.sum().sort_values(ascending=False)

if groups_with_diffs > 0 and not field_diffs_counts.empty:
    print("=== ПОЛЯ, КОТОРЫЕ ЧАЩЕ ВСЕГО РАЗЛИЧАЮТСЯ ===\n")
    for field, count in field_diffs_counts.items():
        if count == 0:
            continue
        print(f"{field}: {count} групп ({count / groups_with_diffs * 100:.2f}%)")

=== АНАЛИЗ РАЗЛИЧИЙ В ПОЛЯХ ДЛЯ ДУБЛИКАТОВ ПО ПОСЛЕДОВАТЕЛЬНОСТЯМ (agab_df) ===

Всего групп дубликатов: 64032

Групп с различиями: 63348 (98.93%)
Групп без различий: 684 (1.07%)

Строк (записей) с различиями: 127356 (98.55%)
Строк (записей) без различий: 1872 (1.45%)

=== ПОЛЯ, КОТОРЫЕ ЧАЩЕ ВСЕГО РАЗЛИЧАЮТСЯ ===

affinity: 63326 групп (99.97%)
dataset: 22 групп (0.03%)


In [65]:
print("=== АНАЛИЗ РАЗБРОСА АФФИНИТЕТА В ГРУППАХ ДУБЛИКАТОВ ===\n")

# Работаем с копией дубликатов
analysis_df = hl_duplicates.copy()

# Преобразуем аффинитет в числа, нечисловые значения станут NaN
analysis_df['affinity_numeric'] = pd.to_numeric(analysis_df['affinity'], errors='coerce')

# Группируем
grouped_analysis = analysis_df.groupby(group_cols, sort=False)

# Считаем размах (max - min) и проверяем единственность типа аффинитета
agg_funcs = {
    'affinity_numeric': lambda x: x.max() - x.min(),
    'affinity_type': 'nunique'
}

group_stats = grouped_analysis.agg(agg_funcs)
group_stats.columns = ['affinity_range', 'affinity_type_count']

# Фильтруем группы, где affinity_numeric удалось посчитать (не NaN)
valid_stats = group_stats.dropna(subset=['affinity_range'])

print(f"Групп с валидными числовыми аффинитетами: {len(valid_stats)}")
print(f"Средний разброс аффинитета в группе: {valid_stats['affinity_range'].mean():.4f}")
print(f"Медианный разброс: {valid_stats['affinity_range'].median():.4f}")
print(f"Максимальный разброс: {valid_stats['affinity_range'].max():.4f}\n")

# Проверка на разные типы аффинитета в одной группе
mixed_types = valid_stats[valid_stats['affinity_type_count'] > 1]
if not mixed_types.empty:
    print(f"ВНИМАНИЕ: В {len(mixed_types)} группах смешаны разные типы аффинитета (сравнение может быть некорректным).")
else:
    print("Типы аффинитета внутри каждой группы совпадают (корректно сравнивать числа).\n")

# Топ-5 групп с самым большим разбросом
print("=== ТОП-5 ГРУПП С САМЫМ БОЛЬШИМ РАЗБРОСОМ АФФИНИТЕТА ===")
top_diff_groups = valid_stats.nlargest(5, 'affinity_range')

for idx, (indices, row) in enumerate(top_diff_groups.iterrows()):
    heavy, light, antigen = indices
    print(f"\nГруппа {idx+1} (разброс {row['affinity_range']:.4f}):")
    # Получаем строки этой группы
    group_rows = analysis_df[
        (analysis_df['heavy_sequence'] == heavy) & 
        (analysis_df['light_sequence'] == light) &
        (analysis_df['antigen_sequence'] == antigen)
    ]
    display(group_rows[['dataset', 'affinity_type', 'affinity', 'confidence', 'processed_measurement']])

=== АНАЛИЗ РАЗБРОСА АФФИНИТЕТА В ГРУППАХ ДУБЛИКАТОВ ===

Групп с валидными числовыми аффинитетами: 64032
Средний разброс аффинитета в группе: 2.8970
Медианный разброс: 3.2739
Максимальный разброс: 3.8345

Типы аффинитета внутри каждой группы совпадают (корректно сравнивать числа).

=== ТОП-5 ГРУПП С САМЫМ БОЛЬШИМ РАЗБРОСОМ АФФИНИТЕТА ===

Группа 1 (разброс 3.8345):


,dataset,affinity_type,affinity,confidence,processed_measurement
1021153,abbd,-log KD,9.834508,high,7.917254
1021154,abbd,-log KD,6.0,high,7.917254



Группа 2 (разброс 3.7936):


,dataset,affinity_type,affinity,confidence,processed_measurement
409471,abbd,-log KD,9.793633,high,7.896816
409472,abbd,-log KD,6.0,high,7.896816



Группа 3 (разброс 3.7556):


,dataset,affinity_type,affinity,confidence,processed_measurement
471033,abbd,-log KD,9.755628,high,7.877814
471034,abbd,-log KD,6.0,high,7.877814



Группа 4 (разброс 3.7512):


,dataset,affinity_type,affinity,confidence,processed_measurement
591881,abbd,-log KD,9.75118,high,7.87559
591882,abbd,-log KD,6.0,high,7.87559



Группа 5 (разброс 3.7379):


,dataset,affinity_type,affinity,confidence,processed_measurement
961707,abbd,-log KD,9.737911,high,7.868956
961708,abbd,-log KD,6.0,high,7.868956


#### Drop duplicates

In [66]:
# Удаляем дубликаты по heavy_sequence + light_sequence + antigen_sequence
# Оставляем первую запись из каждой группы (affinity отличается, но это не важно)

print(f"Размер до удаления дубликатов: {agab_df.shape}")

agab_df = agab_df.drop_duplicates(
    subset=['heavy_sequence', 'light_sequence', 'antigen_sequence'],
    keep='first'
)

print(f"Размер после удаления дубликатов: {agab_df.shape}")

Размер до удаления дубликатов: (402392, 11)
Размер после удаления дубликатов: (337196, 11)


#### ANARCI

In [67]:
def is_empty(x):
    return x is None or (isinstance(x, float) and np.isnan(x)) or (isinstance(x, str) and x.strip() == "")

scfv_df = agab_df[agab_df['scfv'] == True].copy()

# сколько scFv-строк уже имеют обе цепи
both_present = (~scfv_df["heavy_sequence"].apply(is_empty)) & (~scfv_df["light_sequence"].apply(is_empty))
only_heavy   = (~scfv_df["heavy_sequence"].apply(is_empty)) & ( scfv_df["light_sequence"].apply(is_empty))
only_light   = ( scfv_df["heavy_sequence"].apply(is_empty)) & (~scfv_df["light_sequence"].apply(is_empty))

print("scFv всего:", len(scfv_df))
print("оба домена уже раздельно:", both_present.sum())
print("только heavy заполнен:", only_heavy.sum())
print("только light заполнен:", only_light.sum())
print("оба пустые:", ((~both_present) & (~only_heavy) & (~only_light)).sum())

scFv всего: 53603
оба домена уже раздельно: 0
только heavy заполнен: 53603
только light заполнен: 0
оба пустые: 0


In [68]:
from multiprocessing import Pool
import numpy as np
import sys
import os

# Add current directory to path to import worker module
if os.getcwd() not in sys.path:
    sys.path.append(os.getcwd())

# Import worker function from external file to fix pickling error
try:
    from data.anarci_worker import run_anarci_batch
except ImportError:
    # Fallback if running from data directory directly
    try:
        from anarci_worker import run_anarci_batch
    except ImportError:
        print("Error: Could not import anarci_worker. Make sure data/anarci_worker.py exists.")

def process_antibody_sequences_optimized(df, n_jobs=8, batch_size=5000):
    """
    Optimized version of antibody processing using multiprocessing.
    Runs ANARCI in parallel batches and minimizes DataFrame operations.
    """
    df = df.copy()
    
    # 1. Prepare sequences list efficiently
    print("Preparing sequences...")
    sequences_to_anarci = []
    mapping_list = [] 
    
    rows_iter = zip(df.index, df['scfv'], df['heavy_sequence'], df['light_sequence'])
    
    for idx, is_scfv, h_seq, l_seq in tqdm(rows_iter, total=len(df), desc="Collecting sequences"):
        if is_scfv:
            seq = h_seq
            if pd.isna(seq) or len(seq) == 0:
                 seq = l_seq if isinstance(l_seq, str) else ''
            
            if isinstance(seq, str) and len(seq) > 0:
                internal_id = f"{idx}_scfv"
                sequences_to_anarci.append((internal_id, seq))
                mapping_list.append((idx, 'scfv', seq))
        else:
            if isinstance(h_seq, str) and len(h_seq) > 0:
                internal_id = f"{idx}_heavy"
                sequences_to_anarci.append((internal_id, h_seq))
                mapping_list.append((idx, 'heavy', h_seq))
            
            if isinstance(l_seq, str) and len(l_seq) > 0:
                internal_id = f"{idx}_light"
                sequences_to_anarci.append((internal_id, l_seq))
                mapping_list.append((idx, 'light', l_seq))
                
    total_seqs = len(sequences_to_anarci)
    print(f"Running ANARCI on {total_seqs} sequences using {n_jobs} processes...")
    
    # 2. Parallel execution
    chunk_size = min(batch_size, max(1, total_seqs // (n_jobs * 2)))
    chunks = [sequences_to_anarci[i : i + chunk_size] for i in range(0, total_seqs, chunk_size)]
    
    all_numbering = []
    all_alignment_details = []
    
    with Pool(processes=n_jobs) as pool:
        results = list(tqdm(pool.imap(run_anarci_batch, chunks), total=len(chunks), desc="ANARCI Parallel"))
    
    # Unpack results
    for num, aln, err in results:
        if err and not isinstance(err, str): 
             pass
        if num is None:
            print(f"Batch failed: {err}")
            continue 
        all_numbering.extend(num)
        all_alignment_details.extend(aln)

    # 3. Process results efficiently
    print("Processing ANARCI results...")
    
    heavy_seqs = {}
    light_seqs = {}
    heavy_specs = {}
    light_specs = {}
    heavy_germs = {}
    light_germs = {}
    heavy_nums = {}
    light_nums = {}
    
    for i, (num_hits, hits) in tqdm(enumerate(zip(all_numbering, all_alignment_details)), total=len(all_numbering), desc="Parsing results"):
        if not hits or not num_hits: 
            continue
            
        original_idx, seq_type, full_seq = mapping_list[i]
        
        for domain_idx, hit in enumerate(hits):
            if not hit: continue
            
            chain_type = hit.get('chain_type', 'unknown')
            start = hit.get('query_start')
            end = hit.get('query_end')
            species = hit.get('species')
            germlines = hit.get('germlines')
            
            domain_data = num_hits[domain_idx]
            numbering_json = None
            if domain_data:
                residues_json = [
                    {"pos": r[0][0], "ins": r[0][1].strip(), "aa": r[1]}
                    for r in domain_data[0]
                ]
                numbering_obj = {
                    "domain_start": domain_data[1],
                    "domain_end": domain_data[2],
                    "residues": residues_json
                }
                numbering_json = json.dumps([numbering_obj])
            
            domain_seq = full_seq[start : end] if (start is not None and end is not None) else None

            is_heavy = False
            if seq_type == 'scfv':
                if chain_type == 'H': is_heavy = True
                elif chain_type in ['K', 'L']: is_heavy = False
                else: continue
            elif seq_type == 'heavy':
                if chain_type == 'H': is_heavy = True
                else: continue
            elif seq_type == 'light':
                if chain_type in ['K', 'L']: is_heavy = False
                else: continue
            
            if is_heavy:
                heavy_seqs[original_idx] = domain_seq
                heavy_specs[original_idx] = species
                heavy_germs[original_idx] = str(germlines)
                heavy_nums[original_idx] = numbering_json
            else:
                light_seqs[original_idx] = domain_seq
                light_specs[original_idx] = species
                light_germs[original_idx] = str(germlines)
                light_nums[original_idx] = numbering_json

    print("Updating DataFrame columns...")
    
    # Convert dictionaries to Series once
    heavy_seqs_series = pd.Series(heavy_seqs, name='heavy_sequence')
    light_seqs_series = pd.Series(light_seqs, name='light_sequence')
    
    # Use combine_first to update only found values
    # heavy_seqs_series contains only updated sequences, combine_first fills gaps from original
    df['heavy_sequence'] = heavy_seqs_series.combine_first(df['heavy_sequence'])
    df['light_sequence'] = light_seqs_series.combine_first(df['light_sequence'])
    
    # For new columns, map is fine as they are empty or overwritten
    df['heavy_species'] = df.index.map(heavy_specs)
    df['light_species'] = df.index.map(light_specs)
    df['heavy_germlines'] = df.index.map(heavy_germs)
    df['light_germlines'] = df.index.map(light_germs)
    df['heavy_numbering'] = df.index.map(heavy_nums)
    df['light_numbering'] = df.index.map(light_nums)
    found_h = df['heavy_sequence'].notna().sum()
    found_l = df['light_sequence'].notna().sum()
    print(f"Extraction complete. Found {found_h} Heavy chains and {found_l} Light chains.")
    
    return df

# Apply the optimized function
if __name__ == '__main__':
    agab_df = process_antibody_sequences_optimized(agab_df, n_jobs=16)


Preparing sequences...


Running ANARCI on 620789 sequences using 16 processes...


ANARCI Parallel:   0%|          | 0/125 [00:00<?, ?it/s]

Processing ANARCI results...


Parsing results:   0%|          | 0/620789 [00:00<?, ?it/s]

Updating DataFrame columns...
Extraction complete. Found 337196 Heavy chains and 337196 Light chains.


### Проверка ANARCI


### Save to agab.parquet

In [76]:
# Сохраняем датафрейм для использования в других ноутбуках
output_path = 'agab.parquet'

# Fix mixed types for Parquet export
# 'affinity' contains both numbers and strings (e.g. 'h'), so we must save as string
agab_df['affinity'] = agab_df['affinity'].astype(str)
agab_df['processed_measurement'] = agab_df['processed_measurement'].astype(str)

agab_df.to_parquet(output_path, index=False, engine='pyarrow')
print(f"Отфильтрованные данные сохранены в {output_path}")
print(f"Размер сохраненных данных: {agab_df.shape}")


Отфильтрованные данные сохранены в agab.parquet
Размер сохраненных данных: (337196, 17)


In [ ]:
try:
    df = pd.read_parquet('agab.parquet')
except FileNotFoundError:
    print("Файл agab.parquet не найден, использую текущий agab_df из памяти (если есть)")
    if 'agab_df' in locals():
        df = agab_df
    else:
        raise ValueError("Загрузите датафрейм в переменную df")

print(f"Всего записей: {len(df)}")

In [72]:
print("=== ПРОВЕРКА РЕЗУЛЬТАТОВ ОБРАБОТКИ (agab_df_test) ===\n")

# 1. Проверка наличия новых колонок
expected_cols = [
    'heavy_species', 'light_species',
    'heavy_germlines', 'light_germlines',
    'heavy_numbering', 'light_numbering'
]
missing_cols = [c for c in expected_cols if c not in agab_df.columns]
if missing_cols:
    print(f"[ОШИБКА] Отсутствуют колонки: {missing_cols}")
else:
    print(f"[OK] Все новые колонки созданы: {expected_cols}")

# 2. Проверка заполнения (непустые значения)
print("\n--- Статистика заполнения ---")
total_rows = len(agab_df)
if total_rows > 0:
    for col in expected_cols + ['heavy_sequence', 'light_sequence']:
        filled = agab_df[col].notna().sum()
        print(f"{col}: {filled}/{total_rows} ({filled/total_rows*100:.1f}%)")
else:
    print("Датафрейм пуст!")

# 3. Проверка формата JSON в numbering
print("\n--- Проверка формата JSON (первые 3 непустых) ---")
sample_h = agab_df[agab_df['heavy_numbering'].notna()].head(3)
for idx, row in sample_h.iterrows():
    try:
        data = json.loads(row['heavy_numbering'])
        print(f"[OK] Row {idx} heavy_numbering valid JSON. Domains: {len(data)}")
        if data:
             print(f"     First domain residues count: {len(data[0].get('residues', []))}")
    except Exception as e:
        print(f"[ОШИБКА] Row {idx} heavy_numbering invalid JSON: {e}")

# 4. Проверка разделения scFv
print("\n--- Проверка разделения scFv ---")
scfv_rows = agab_df[agab_df['scfv'] == True]
if len(scfv_rows) > 0:
    scfv_with_chains = scfv_rows[
        scfv_rows['heavy_sequence'].notna() & 
        scfv_rows['light_sequence'].notna()
    ]
    scfv_only_heavy = scfv_rows[
        scfv_rows['heavy_sequence'].notna() & 
        scfv_rows['light_sequence'].isna()
    ]
    scfv_only_light = scfv_rows[
        scfv_rows['heavy_sequence'].isna() & 
        scfv_rows['light_sequence'].notna()
    ]
    print(f"Всего scFv: {len(scfv_rows)}")
    print(f"scFv с обеими цепями (H+L): {len(scfv_with_chains)} ({len(scfv_with_chains)/len(scfv_rows)*100:.1f}%)")
    print(f"scFv только с Heavy: {len(scfv_only_heavy)} ({len(scfv_only_heavy)/len(scfv_rows)*100:.1f}%)")
    print(f"scFv только с Light: {len(scfv_only_light)} ({len(scfv_only_light)/len(scfv_rows)*100:.1f}%)")
else:
    print("В датафрейме нет записей scFv.")

# 5. Пример данных
print("\n--- Пример одной записи (Heavy) ---")
if not sample_h.empty:
    example_row = sample_h.iloc[0]
    print(f"Species: {example_row['heavy_species']}")
    print(f"Germlines: {example_row['heavy_germlines']}")
    print(f"Sequence (trimmed): {example_row['heavy_sequence'][:20]}...")
else:
    print("Нет обработанных heavy chains для показа примера.")


=== ПРОВЕРКА РЕЗУЛЬТАТОВ ОБРАБОТКИ (agab_df_test) ===

[OK] Все новые колонки созданы: ['heavy_species', 'light_species', 'heavy_germlines', 'light_germlines', 'heavy_numbering', 'light_numbering']

--- Статистика заполнения ---
heavy_species: 337196/337196 (100.0%)
light_species: 337196/337196 (100.0%)
heavy_germlines: 337196/337196 (100.0%)
light_germlines: 337196/337196 (100.0%)
heavy_numbering: 337196/337196 (100.0%)
light_numbering: 337196/337196 (100.0%)
heavy_sequence: 337196/337196 (100.0%)
light_sequence: 337196/337196 (100.0%)

--- Проверка формата JSON (первые 3 непустых) ---
[OK] Row 3 heavy_numbering valid JSON. Domains: 1
     First domain residues count: 128
[OK] Row 5 heavy_numbering valid JSON. Domains: 1
     First domain residues count: 128
[OK] Row 12 heavy_numbering valid JSON. Domains: 1
     First domain residues count: 128

--- Проверка разделения scFv ---
Всего scFv: 53603
scFv с обеими цепями (H+L): 53603 (100.0%)
scFv только с Heavy: 0 (0.0%)
scFv только с Li

In [ ]:
# Функция для безопасного извлечения длины
def get_len(seq):
    return len(seq) if isinstance(seq, str) else 0

df['heavy_len'] = df['heavy_sequence'].apply(get_len)
df['light_len'] = df['light_sequence'].apply(get_len)

# --- ПРОВЕРКА 1: Распределение длин ---
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Heavy chain length
h_lens = df[df['heavy_len'] > 0]['heavy_len']
sns.histplot(h_lens, bins=range(80, 160), ax=axes[0], color='skyblue')
axes[0].set_title(f'Heavy Chain Lengths (Mean: {h_lens.mean():.1f})')
axes[0].axvline(105, color='r', linestyle='--', alpha=0.5, label='Min Exp (105)')
axes[0].axvline(135, color='r', linestyle='--', alpha=0.5, label='Max Exp (135)')
axes[0].legend()

# Light chain length
l_lens = df[df['light_len'] > 0]['light_len']
sns.histplot(l_lens, bins=range(80, 160), ax=axes[1], color='lightgreen')
axes[1].set_title(f'Light Chain Lengths (Mean: {l_lens.mean():.1f})')
axes[1].axvline(100, color='r', linestyle='--', alpha=0.5, label='Min Exp (100)')
axes[1].axvline(125, color='r', linestyle='--', alpha=0.5, label='Max Exp (125)')
axes[1].legend()

plt.tight_layout()
plt.show()

# Выбросы
print("\n=== Выбросы по длине ===")
print(f"Heavy < 90 aa: {len(h_lens[h_lens < 90])} шт.")
print(f"Heavy > 140 aa: {len(h_lens[h_lens > 140])} шт.")
print(f"Light < 90 aa: {len(l_lens[l_lens < 90])} шт.")
print(f"Light > 135 aa: {len(l_lens[l_lens > 135])} шт.")

# --- ПРОВЕРКА 2: Start/End паттерны ---
def get_top_kmers(sequences, k=5, n=10, side='start'):
    seqs = [s for s in sequences if isinstance(s, str) and len(s) >= k]
    if side == 'start':
        kmers = [s[:k] for s in seqs]
    else:
        kmers = [s[-k:] for s in seqs]
    return Counter(kmers).most_common(n)

print("\n=== Топ-10 паттернов начала и конца (Heavy) ===")
print("Start:", get_top_kmers(df['heavy_sequence'], side='start'))
print("End:  ", get_top_kmers(df['heavy_sequence'], side='end'))

print("\n=== Топ-10 паттернов начала и конца (Light) ===")
print("Start:", get_top_kmers(df['light_sequence'], side='start'))
print("End:  ", get_top_kmers(df['light_sequence'], side='end'))

# --- ПРОВЕРКА 3: Консервативные цистеины (из numbering) ---
print("\n=== Проверка консервативных цистеинов (C23, C104) ===")

def check_cysteines(numbering_json):
    if not numbering_json or pd.isna(numbering_json):
        return None
    try:
        data = json.loads(numbering_json)
        if not data: return False
        
        # Получаем список остатков первого домена
        residues = data[0].get('residues', [])
        
        has_c23 = False
        has_c104 = False
        
        for r in residues:
            # r['pos'] это номер позиции IMGT
            # r['aa'] это аминокислота
            pos = r.get('pos')
            aa = r.get('aa')
            
            if pos == 23 and aa == 'C':
                has_c23 = True
            if pos == 104 and aa == 'C':
                has_c104 = True
                
        return has_c23 and has_c104
    except:
        return None

df['heavy_cys_ok'] = df['heavy_numbering'].apply(check_cysteines)
df['light_cys_ok'] = df['light_numbering'].apply(check_cysteines)

print(f"Heavy Chains with both C23 & C104: {df['heavy_cys_ok'].sum()} / {df['heavy_numbering'].notna().sum()}")
print(f"Light Chains with both C23 & C104: {df['light_cys_ok'].sum()} / {df['light_numbering'].notna().sum()}")

# --- ПРОВЕРКА 4: Зависимость от вида (Species) ---
print("\n=== Средняя длина по видам (Top 5) ===")
top_species = df['heavy_species'].value_counts().head(5).index
print(df[df['heavy_species'].isin(top_species)].groupby('heavy_species')['heavy_len'].agg(['mean', 'std', 'count']))